In [1]:
pip install pandas numpy matplotlib seaborn openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_california_nashville_migration(acs_file, nashville_migration_file, updated_net_file, output_file=None):
    """
    Analyze migration data from California to Nashville across multiple datasets.
    
    Parameters:
    -----------
    acs_file : str
        Path to the ACS Migration Excel file
    nashville_migration_file : str
        Path to the Nashville Migration Excel file
    updated_net_file : str
        Path to the Updated Net Excel file
    output_file : str, optional
        Path to save combined results as CSV
        
    Returns:
    --------
    dict
        Dictionary containing combined migration data and analysis results
    """
    # Store results and data
    results = {
        'acs_data': None,
        'nashville_data': None,
        'updated_net_data': None,
        'combined_data': None,
        'summary': None
    }
    
    # Load ACS Migration data
    try:
        print("Loading ACS Migration data...")
        acs_df = pd.read_excel(acs_file)
        
        # Based on the image, we know the exact column names
        ca_column = "State of Residence 1 Year Ago"  # Column F in the image
        movers_column = "Movers from State of Previous Residence to County of Current Residence1 Estimate"  # Column G in the image
        county_column = "County of Current Residence"  # Column B in the image
        
        # Check if columns exist, if not try partial matching
        if ca_column not in acs_df.columns:
            # Try to find the closest match
            ca_column = [col for col in acs_df.columns if 'State' in col and 'Residence' in col and 'Year Ago' in col][0]
            print(f"Using '{ca_column}' as State of Residence 1 Year Ago column")
            
        if movers_column not in acs_df.columns:
            # Try to find the closest match for movers column
            movers_column = [col for col in acs_df.columns if 'Movers' in col and 'Estimate' in col][0]
            print(f"Using '{movers_column}' as Movers Estimate column")
            
        if county_column not in acs_df.columns:
            # Try to find the closest match for county column
            county_column = [col for col in acs_df.columns if 'County' in col and 'Current' in col][0]
            print(f"Using '{county_column}' as County of Current Residence column")
            
        # Filter for California data
        ca_acs_data = acs_df[acs_df[ca_column] == 'California']
        
        if len(ca_acs_data) == 0:
            # Try case-insensitive matching as a fallback
            ca_acs_data = acs_df[acs_df[ca_column].str.lower() == 'california']
            
        # Debug info
        print(f"ACS columns: {acs_df.columns.tolist()}")
        print(f"Unique values in {ca_column}: {acs_df[ca_column].unique().tolist()}")
            
        results['acs_data'] = ca_acs_data
        
        print(f"Found {len(ca_acs_data)} entries for California in ACS data")
        if len(ca_acs_data) > 0:
            print("Sample California migration data:")
            print(ca_acs_data[[county_column, ca_column, movers_column]].head())
    except Exception as e:
        print(f"Error processing ACS data: {e}")
        import traceback
        traceback.print_exc()
        results['acs_data'] = pd.DataFrame()
    
    # Load Nashville Migration data
    try:
        print("Loading Nashville Migration data...")
        nashville_df = pd.read_excel(nashville_migration_file)
        
        # Filter for California metro areas
        metro_b_column = [col for col in nashville_df.columns if 'Metro' in col and 'Geography B' in col][0]
        flow_column = [col for col in nashville_df.columns if 'Flow from Geography B' in col][0]
        net_column = [col for col in nashville_df.columns if 'Net Migration' in col][0]
        
        ca_nashville_data = nashville_df[nashville_df[metro_b_column].str.contains('CA Metro Area', na=False)]
        results['nashville_data'] = ca_nashville_data
        
        print(f"Found {len(ca_nashville_data)} California metro areas in Nashville Migration data")
    except Exception as e:
        print(f"Error processing Nashville Migration data: {e}")
        results['nashville_data'] = pd.DataFrame()
    
    # Load Updated Net data
    try:
        print("Loading Updated Net data...")
        updated_net_df = pd.read_excel(updated_net_file)
        
        # Get column names
        city_column = 'City' if 'City' in updated_net_df.columns else None
        path_id_column = 'Path_ID' if 'Path_ID' in updated_net_df.columns else None
        net_migration_column = [col for col in updated_net_df.columns if 'Net Migration' in col][0]
        
        print(f"Updated Net columns: {updated_net_df.columns.tolist()}")
        
        # List of major California cities and locations to identify
        california_cities = [
            'Los Angeles', 'San Francisco', 'San Diego', 'Sacramento', 'San Jose', 
            'Oakland', 'Fresno', 'Long Beach', 'Bakersfield', 'Anaheim', 'Santa Ana',
            'Riverside', 'Stockton', 'Chula Vista', 'Irvine', 'Fremont', 'San Bernardino',
            'Modesto', 'Fontana', 'Oxnard', 'Moreno Valley', 'Huntington Beach', 'Glendale',
            'Santa Clarita', 'Santa Rosa', 'Ontario', 'Rancho Cucamonga', 'Oceanside',
            'Santa Barbara', 'Palo Alto', 'Berkeley', 'Napa', 'Ventura', 'Sonoma',
            'Monterey', 'Newport Beach', 'Malibu', 'La Jolla', 'Palm Springs',
            'Laguna Beach', 'Santa Monica', 'Silicon Valley', 'Bay Area', 
            'Pasadena', 'Redondo Beach', 'Burbank', 'Beverly Hills', 'Hollywood',
            'Marin', 'Orange County', 'Yuba City', 'Merced', 'Salinas'
        ]
        
        # Function to check if Path_ID contains a California location
        def is_california_location(path_id):
            if not isinstance(path_id, str):
                return False
                
            # If explicitly marked as CA, it's from California
            if 'CA' in path_id:
                return True
                
            # Check for known California cities
            for city in california_cities:
                # Match only the city portion after "Nashville_"
                if path_id.startswith("Nashville_") and city in path_id.split("Nashville_")[1]:
                    return True
                    
            return False
        
        # Filter for paths that represent California migration to Nashville
        if path_id_column:
            # Apply the filter to find California paths
            ca_filter = updated_net_df[path_id_column].apply(is_california_location)
            
            # Get entries where positive net migration means inflow to Nashville
            inflow_filter = updated_net_df[net_migration_column] > 0
            
            # Combine filters to get California inflow to Nashville
            ca_updated_net_data = updated_net_df[ca_filter & inflow_filter]
            
            # Debug info
            print(f"Found {len(ca_updated_net_data)} California inflow entries in Updated Net data")
            if not ca_updated_net_data.empty:
                print("Sample California inflow data:")
                print(ca_updated_net_data[[path_id_column, net_migration_column]].head())
                
            results['updated_net_data'] = ca_updated_net_data
        else:
            print("Path_ID column not found in Updated Net data")
            results['updated_net_data'] = pd.DataFrame()
    except Exception as e:
        print(f"Error processing Updated Net data: {e}")
        import traceback
        traceback.print_exc()
        results['updated_net_data'] = pd.DataFrame()
    
    # Combine data sources
    print("Combining data sources...")
    combined_data = []
    
    # Process ACS data
    if not results['acs_data'].empty:
        county_column = "County of Current Residence"
        movers_column = "Movers from State of Previous Residence to County of Current Residence1 Estimate"
        moe_column = "Movers from State of Previous Residence to County of Current Residence1MOE"
        
        # Check if columns exist, if not try partial matching
        if county_column not in results['acs_data'].columns:
            county_column = [col for col in results['acs_data'].columns if 'County' in col and 'Current' in col][0]
        
        if movers_column not in results['acs_data'].columns:
            movers_column = [col for col in results['acs_data'].columns if 'Movers' in col and 'Estimate' in col][0]
        
        # Process each county entry for California migration
        for _, row in results['acs_data'].iterrows():
            county = row.get(county_column, "Unknown County")
            movers = row.get(movers_column, 0)
            
            # Only include rows with actual migration numbers
            if movers > 0:
                combined_data.append({
                    'source_dataset': 'ACS Migration',
                    'geography': 'California',
                    'subgeography': 'State',
                    'destination': county,
                    'inflow_count': movers,
                    'confidence': 'Medium'  # ACS data has margins of error
                })
                
        print(f"Added {len([d for d in combined_data if d['source_dataset'] == 'ACS Migration'])} records from ACS data")
    
    # Process Nashville Migration data
    if not results['nashville_data'].empty:
        for _, row in results['nashville_data'].iterrows():
            metro_area = row.get(metro_b_column, '')
            city = metro_area.split(',')[0] if ',' in metro_area else metro_area
            inflow = row.get(flow_column, 0)
            
            # Clean up metro area name
            if 'Metro Area' in city:
                city = city.replace(' Metro Area', '')
            
            combined_data.append({
                'source_dataset': 'Nashville Migration',
                'geography': 'California',
                'subgeography': city,
                'destination': 'Nashville Metro',
                'inflow_count': inflow,
                'confidence': 'High'  # Metro-to-metro data likely more reliable
            })
    
    # Process Updated Net data
    if not results['updated_net_data'].empty:
        path_id_column = 'Path_ID' if 'Path_ID' in results['updated_net_data'].columns else 'City'
        net_migration_column = [col for col in results['updated_net_data'].columns if 'Net Migration' in col][0]
        
        # California cities map to help with identification
        california_cities_map = {
            'Los Angeles': 'Los Angeles', 'LA': 'Los Angeles',
            'San Francisco': 'San Francisco', 'SF': 'San Francisco',
            'San Diego': 'San Diego', 'Sacramento': 'Sacramento',
            'San Jose': 'San Jose', 'Oakland': 'Oakland',
            'Fresno': 'Fresno', 'Long Beach': 'Long Beach',
            'Bakersfield': 'Bakersfield', 'Anaheim': 'Anaheim',
            'Santa Ana': 'Santa Ana', 'Riverside': 'Riverside',
            'Stockton': 'Stockton', 'Chula Vista': 'Chula Vista',
            'Fremont': 'Fremont', 'San Bernardino': 'San Bernardino',
            'Modesto': 'Modesto', 'Fontana': 'Fontana',
            'Oxnard': 'Oxnard', 'Moreno Valley': 'Moreno Valley',
            'Glendale': 'Glendale', 'Santa Clarita': 'Santa Clarita',
            'Santa Rosa': 'Santa Rosa', 'Ontario': 'Ontario',
            'Oceanside': 'Oceanside', 'Santa Barbara': 'Santa Barbara',
            'Palo Alto': 'Palo Alto', 'Berkeley': 'Berkeley',
            'Napa': 'Napa', 'Ventura': 'Ventura',
            'Monterey': 'Monterey', 'Newport': 'Newport Beach',
            'Malibu': 'Malibu', 'Pasadena': 'Pasadena',
            'Burbank': 'Burbank', 'Hollywood': 'Hollywood',
            'Orange County': 'Orange County', 'Irvine': 'Irvine',
            'Yuba City': 'Yuba City', 'Merced': 'Merced',
            'Salinas': 'Salinas', 'Santa Monica': 'Santa Monica',
            'Silicon Valley': 'Silicon Valley', 'Bay Area': 'Bay Area'
        }
        
        for _, row in results['updated_net_data'].iterrows():
            path_id = row.get(path_id_column, '')
            migration = row.get(net_migration_column, 0)
            
            # Extract city name from Path_ID (format: "Nashville_CityName")
            if isinstance(path_id, str) and 'Nashville_' in path_id:
                city_part = path_id.split('Nashville_')[1]
                # Remove state code if present
                if ' ' in city_part and city_part.split(' ')[-1] in ['CA', 'TX', 'NY', 'FL', 'OH', 'GA', 'TN', 'IL', 'MI']:
                    city = ' '.join(city_part.split(' ')[:-1])
                else:
                    city = city_part
            else:
                city = str(path_id).replace('CA', '').strip()
            
            # Try to map to a standard California city name
            for key in california_cities_map:
                if key in city:
                    city = california_cities_map[key]
                    break
            
            combined_data.append({
                'source_dataset': 'Updated Net',
                'geography': 'California',
                'subgeography': city,
                'destination': 'Nashville',
                'inflow_count': abs(migration),  # We know these are positive values for inflow
                'confidence': 'High'  # Direct city-to-city connections
            })
            
        print(f"Added {len([d for d in combined_data if d['source_dataset'] == 'Updated Net'])} records from Updated Net data")
    
    # Convert to DataFrame
    results['combined_data'] = pd.DataFrame(combined_data)
    
    # Generate summary statistics
    if not results['combined_data'].empty:
        # Total inflow from California
        total_inflow = results['combined_data']['inflow_count'].sum()
        
        # Group by subgeography (cities/metros within CA)
        subgeo_summary = results['combined_data'].groupby('subgeography')['inflow_count'].sum().reset_index()
        subgeo_summary = subgeo_summary.sort_values('inflow_count', ascending=False)
        
        # Group by source dataset
        dataset_summary = results['combined_data'].groupby('source_dataset')['inflow_count'].sum().reset_index()
        
        results['summary'] = {
            'total_inflow_from_california': total_inflow,
            'top_source_areas': subgeo_summary.to_dict(orient='records'),
            'dataset_contributions': dataset_summary.to_dict(orient='records')
        }
        
        print(f"Total inflow from California: {total_inflow}")
        print("Top 5 source areas within California:")
        print(subgeo_summary.head(5))
    
    # Save combined data if output file specified
    if output_file and not results['combined_data'].empty:
        results['combined_data'].to_csv(output_file, index=False)
        print(f"Combined data saved to {output_file}")
    
    return results

def visualize_california_nashville_migration(results):
    """
    Create visualizations for the California to Nashville migration data.
    
    Parameters:
    -----------
    results : dict
        Dictionary containing the migration analysis results
    """
    combined_data = results.get('combined_data')
    if combined_data is None or combined_data.empty:
        print("No data available for visualization")
        return
    
    # Set up the visualization style
    plt.style.use('seaborn-v0_8-whitegrid')
    
    # 1. Top source areas within California
    plt.figure(figsize=(12, 8))
    top_sources = combined_data.groupby('subgeography')['inflow_count'].sum().nlargest(10).reset_index()
    
    sns.barplot(data=top_sources, x='inflow_count', y='subgeography', palette='viridis')
    plt.title('Top 10 California Areas with Migration to Nashville', fontsize=16)
    plt.xlabel('Number of Migrants', fontsize=14)
    plt.ylabel('California Area', fontsize=14)
    plt.tight_layout()
    plt.savefig('top_california_sources.png')
    plt.close()
    
    # 2. Comparison of data sources
    plt.figure(figsize=(10, 6))
    source_comparison = combined_data.groupby('source_dataset')['inflow_count'].sum().reset_index()
    
    sns.barplot(data=source_comparison, x='source_dataset', y='inflow_count', palette='Set2')
    plt.title('California to Nashville Migration by Data Source', fontsize=16)
    plt.xlabel('Data Source', fontsize=14)
    plt.ylabel('Total Migration Count', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('data_source_comparison.png')
    plt.close()
    
    # 3. Migration counts by confidence level
    plt.figure(figsize=(10, 6))
    confidence_data = combined_data.groupby('confidence')['inflow_count'].sum().reset_index()
    
    sns.barplot(data=confidence_data, x='confidence', y='inflow_count', palette='Blues_d')
    plt.title('California to Nashville Migration by Confidence Level', fontsize=16)
    plt.xlabel('Confidence Level', fontsize=14)
    plt.ylabel('Total Migration Count', fontsize=14)
    plt.tight_layout()
    plt.savefig('migration_by_confidence.png')
    plt.close()
    
    print("Visualizations created and saved as PNG files")

def combined_migration_analysis(acs_file, nashville_migration_file, updated_net_file, output_file=None):
    """
    Complete analysis of California to Nashville migration across all datasets.
    
    Parameters:
    -----------
    acs_file : str
        Path to the ACS Migration Excel file
    nashville_migration_file : str
        Path to the Nashville Migration Excel file
    updated_net_file : str
        Path to the Updated Net Excel file
    output_file : str, optional
        Path to save combined results as CSV
    """
    print("Starting California to Nashville Migration Analysis")
    print("="*50)
    
    # Run the analysis
    results = analyze_california_nashville_migration(
        acs_file=acs_file,
        nashville_migration_file=nashville_migration_file,
        updated_net_file=updated_net_file,
        output_file=output_file
    )
    
    # Create visualizations
    if results['combined_data'] is not None and not results['combined_data'].empty:
        print("\nGenerating visualizations...")
        visualize_california_nashville_migration(results)
        
        # Display the average migration values
        if results['summary']:
            print("\nSummary Statistics:")
            print(f"Total inflow from California: {results['summary']['total_inflow_from_california']}")
            
            # Calculate average migration per data source
            if 'dataset_contributions' in results['summary']:
                datasets = len(results['summary']['dataset_contributions'])
                if datasets > 1:
                    total = sum(item['inflow_count'] for item in results['summary']['dataset_contributions'])
                    average = total / datasets
                    print(f"Average migration count across {datasets} datasets: {average:.2f}")
    
    print("\nAnalysis complete!")
    return results



In [3]:
# Replace these with your actual file paths
acs_file = "ACS_Migration.xlsx"
nashville_migration_file = "Nashville Migration.xlsx"
updated_net_file = "Updated Net.xlsx"
output_file = "california_nashville_migration_combined.csv"

# Run the analysis
results = combined_migration_analysis(
    acs_file=acs_file,
    nashville_migration_file=nashville_migration_file,
    updated_net_file=updated_net_file,
    output_file=output_file
)

Starting California to Nashville Migration Analysis
Loading ACS Migration data...
ACS columns: ['State of Current Residence', 'County of Current Residence', 'State of Residence 1 Year Ago', 'Movers from State of Previous Residence to County of Current Residence1 Estimate', 'Movers from State of Previous Residence to County of Current Residence1MOE']
Unique values in State of Residence 1 Year Ago: ['California', 'Florida', 'Kentucky', 'Tennessee', 'Virginia', 'Africa', 'Europe', 'Alabama', 'Arizona', 'Georgia', 'Illinois', 'Indiana', 'Louisiana', 'Maryland', 'Michigan', 'Mississippi', 'Missouri', 'North Carolina', 'Ohio', 'South Carolina', 'Texas', 'Wisconsin', 'Central America', 'Caribbean', 'Arkansas', 'Colorado', 'Delaware', 'District of Columbia', 'Hawaii', 'Idaho', 'Iowa', 'Kansas', 'Maine', 'Massachusetts', 'Minnesota', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Dakota', 'Utah', 'Vermont'

C:\Users\CooperChristian\AppData\Local\Temp\ipykernel_11728\3476485371.py:344: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=top_sources, x='inflow_count', y='subgeography', palette='viridis')
C:\Users\CooperChristian\AppData\Local\Temp\ipykernel_11728\3476485371.py:356: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=source_comparison, x='source_dataset', y='inflow_count', palette='Set2')
C:\Users\CooperChristian\AppData\Local\Temp\ipykernel_11728\3476485371.py:369: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=confidence_data,

Visualizations created and saved as PNG files

Summary Statistics:
Total inflow from California: 15366
Average migration count across 3 datasets: 5122.00

Analysis complete!
